In [39]:
# Import necessary libraries
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from scipy import signal

In [40]:
class Pixel():
    '''
    Class for keeping pixel cooridinate
    '''

    def __init__(self, kp):
        self.x = int(kp.pt[0])
        self.y = int(kp.pt[1])
        self.point = (self.x, self.y)

    def __repr__(self):
        return f'({self.x}, {self.y})'

    def adjust_width(self, width):
        x = self.x + width
        return (x, self.y)

In [41]:
class SIFT():
    '''
    Class for implimenting SIFT.
    '''
    
    def __init__(self):
        pass

    def extract_feature(self, image):
        gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        features = cv.xfeatures2d.SIFT_create()
        kp, des = features.detectAndCompute(gray_image, None)
        
        return [kp, des]

    def find_pairs(self, image1, image2, theshore):
        kp1, des1 = self.extract_feature(image1)
        kp2, des2 = self.extract_feature(image2)

        if len(kp1) < len(kp2):
            kpA = kp1
            kpB = kp2
            desA = des1
            desB = des2
            imageA = image1
            imageB = image2
        else:
            kpA = kp2
            kpB = kp1
            desA = des2
            desB = des1
            imageA = image2
            imageB = image1

        pairs = []
        for inx_a , des_a in enumerate(desA):
            min_distance = 1000000000
            best_candidate = None
            for inx_b , des_b in enumerate(desB):
                distance = np.linalg.norm(des_a - des_b)
                if distance < min_distance:
                    min_distance = distance
                    best_candidate = inx_b
            if min_distance < theshore:
                pairs.append((Pixel(kpA[inx_a]), Pixel(kpB[best_candidate])))
        
        return [pairs, imageA, imageB]


In [42]:
class Image(SIFT):
    ''' 
    Class for store images and related parameters.
    '''

    FILEPATH = 'ece661_pics\\hw4_image\\'
    FILETYPE = '.png'

    def __init__(self, file1, file2, savename, theshore):
        self.file1 = file1
        self.file2 = file2
        self.load_images()
        self.savename = savename
        self.theshore = theshore

    def load_images(self):
        self.image1 = cv.imread(self.FILEPATH + self.file1)
        self.image2 = cv.imread(self.FILEPATH + self.file2)
        self.image = cv.hconcat([self.image1, self.image2])

    def show_image(self, image=0):
        if image == 1:
            plt.imshow(cv.cvtColor(self.image1, cv.COLOR_BGR2RGB))
        elif image == 2:
            plt.imshow(cv.cvtColor(self.image2, cv.COLOR_BGR2RGB))
        elif image == 0:
            plt.imshow(cv.cvtColor(self.image, cv.COLOR_BGR2RGB))

    def process_sift(self):
        [pairs, imageA, imageB] = SIFT.find_pairs(self, self.image1, self.image2, self.theshore)
        image = self.mark_pairs(pairs, imageA, imageB)
        plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
        savename = f'{self.FILEPATH}{self.savename}_sift{self.FILETYPE}'
        cv.imwrite(savename, image.astype(np.int))

    def mark_pairs(self, pairs, imageA, imageB):
        radius = 2
        thickness = 1
        image = cv.hconcat([imageA, imageB])
        w = imageA.shape[1]
        for pair in pairs:
            color = (0, 0, 255)
            cv.circle(image, pair[0].point, radius, color, thickness)
            cv.circle(image, pair[1].adjust_width(w), radius, color, thickness)
            cv.line(image, pair[0].point, pair[1].adjust_width(w), color, thickness)

        return image


In [56]:
images = [
    Image('pair1\\1.jpg', 'pair1\\2.jpg', 'hw4_task2_1', 100),
    Image('pair2\\1.jpg', 'pair2\\2.jpg', 'hw4_task2_2', 150),
    Image('pair3\\1.jpg', 'pair3\\2.jpg', 'hw4_task2_3', 80),
    Image('pair4\\1.jpg', 'pair4\\2.jpg', 'hw4_task2_4', 100),
    Image('pair5\\1.jpg', 'pair5\\2.jpg', 'hw4_task2_5', 200),
    Image('pair6\\1.jpg', 'pair6\\2.jpg', 'hw4_task2_6', 30)]

In [23]:
for image in images:
    image.process_sift()

KeyboardInterrupt: 